使用单一的优化器调整策略

In [ ]:
from torch.optim import SGD
from mmengine.runner import Runner
from mmengine.optim.scheduler import MultiStepLR
import torch.nn as nn

model=nn.Linear(10,10)
optimizer=SGD(model.parameters(), lr=0.01, momentum=0.9)
param_scheduler=MultiStepLR(optimizer, by_epoch=True,milestones=[8,11], gamma=0.1)
runner=Runner(model, optim_wrapper=dict(type='OptimiWrapper',optimizer=optimizer),
              param_scheduler=param_scheduler)

by_epoch=True指示了学习率按照epoch更新，by_epoch=False表示按照iter更新，同时后面的milestones也对应epoch或者iter
ga'mma=0.1表示学习率衰减为原来的0.1倍。
使用iter更新学习率会更加平滑，但是epoch更新学习率更加直观，方便调整。
可以传入by_epoch=True,让他自动转换为iter的更新方式

In [ ]:
#手动配置
epoch_length=len(train_dataloader)#表示一个epoch需要迭代的iter次数
param_scheduler=MultiStepLR.build_iter_from_epoch(
    optimizer,
    by_epoch=True,
    milestones=[8,11],
    gamma=0.1
    epoch_length=epoch_length
)
#配置文件写法
param_scheduler=dict(type='MultiStepLR',
                     by_epoch=True,
                     milestones=[8,11],
                     gamma=0.1,
                     convert_to_iter_based=True)

组合多个调度策略

In [ ]:
#配置文件写法，直接将多个调度策略的配置组合成列表即可
param_scheduler = [
    # 线性学习率预热调度器
    dict(type='LinearLR',
         start_factor=0.001,
         by_epoch=False,  # 按迭代更新学习率
         begin=0,
         end=50),  # 预热前 50 次迭代
    # 主学习率调度器
    dict(type='MultiStepLR',
         by_epoch=True,  # 按轮次更新学习率
         milestones=[8, 11],
         gamma=0.1)
]
#注意这里增加了 begin 和 end 参数，这两个参数指定了调度器的生效区间。生效区间通常只在多个调度器组合时才需要去设置，
# 使用单个调度器时可以忽略。当指定了 begin 和 end 参数时，表示该调度器只在 [begin, end) 区间内生效，
# 其单位是由 by_epoch 参数决定

# TODO 手动配置写法，不知道用哪个类

动量调度策略

In [ ]:
#配置文件写法，只需要将LR换成Momentum即可
param_scheduler = [
    # the lr scheduler
    dict(type='LinearLR', ...),
    # 动量调度器
    dict(type='LinearMomentum',
         start_factor=0.001,
         by_epoch=False,
         begin=0,
         end=1000)
]

通用的参数调度器策略

In [ ]:
#配置文件写法：将某类LR调度器的名字替换成Param即可
param_scheduler = [
    dict(type='LinearParamScheduler',
         param_name='lr',  # 调度 `optimizer.param_groups` 中名为 'lr' 的变量
         start_factor=0.001,
         by_epoch=False,
         begin=0,
         end=1000)
]